### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv, find_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv('KEYZ.env')
KEY = os.getenv('KEY')


### CME Data

In [4]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
# https://api.nasa.gov/DONKI/CME?startDate=yyyy-MM-dd&endDate=yyyy-MM-dd&api_key=DEMO_KEY
query_url = f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={KEY}"


In [5]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(query_url)
cme_response

<Response [200]>

In [6]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [7]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[0], indent=4))

{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [8]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)
# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[['activityID', 'startTime', 'linkedEvents']]
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [9]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df_cleaned = cme_df.dropna(subset=['linkedEvents'])
cme_df_cleaned.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [10]:
expanded_rows = []

for i in cme_df_cleaned.index:
    activityID=cme_df_cleaned.loc[i,'activityID']
    startTime=cme_df_cleaned.loc[i,'startTime']
    linkedEvents=cme_df_cleaned.loc[i,'linkedEvents'] 
    
    for event in linkedEvents:
        expanded_rows.append({
            'activityID' : activityID,
            'startTime' : startTime,
            'linkedEvents' : event})
        
expanded_df = pd.DataFrame(expanded_rows)
expanded_df

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}
...,...,...,...
1709,2024-04-24T01:48:00-CME-001,2024-04-24T01:48Z,{'activityID': '2024-04-26T00:17:00-IPS-001'}
1710,2024-04-24T15:05:00-CME-001,2024-04-24T15:05Z,{'activityID': '2024-04-24T14:02:00-FLR-001'}
1711,2024-04-25T18:24:00-CME-001,2024-04-25T18:24Z,{'activityID': '2024-04-25T17:03:00-FLR-001'}
1712,2024-04-30T00:36:00-CME-001,2024-04-30T00:36Z,{'activityID': '2024-04-30T00:46:00-FLR-001'}


In [11]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# # and adds the elements individually to a list of dictionaries where each row is one element 

# # Initialize an empty list to store the expanded rows
# expanded_rows = []

# # Iterate over each index in the DataFrame
# for index, row in cme_df_cleaned.iterrows():
#     # Iterate over each dictionary in the list
#      for event in row['linkedEvents']:
#         # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
#         expanded_rows.append({
#             'activityID': row['activityID'],
#             'startTime': row['startTime'],
#             'linkedEventID': event['activityID']  # Extract the linked event's activityID
#         })
# # Create a new DataFrame from the expanded rows
# expanded_df = pd.DataFrame(expanded_rows)
# expanded_df.head()


In [12]:
def extract_activityID_from_dict(dict):
    try:
        return dict['activityID']
    except (KeyError, TypeError) as error:
        print(f"Error: processing the dictionary: {dict}")
        return None

expanded_df['linkedEvents'][1] 
extract_activityID_from_dict(expanded_df['linkedEvents'][1])
        

'2013-05-07T04:37:00-IPS-001'

In [13]:
# # Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# # and verify below that it works as expected using one row from linkedEvents as an example
# # Be sure to use a try and except block to handle errors
# def extract_activityID_from_dict(event_dict):
#     try:
#         return event_dict['']
#     except KeyError as e:
#         print(f"KeyError: {e} - 'activityID' not found in the dictionary: {event_dict}")
#         return None  # Return None if the key is missing
#     except TypeError as e:
#         # Log or print the error if the input is not a valid dictionary
#         print(f"TypeError: {e} - Input is not a valid dictionary: {event_dict}")
#         return None

# example_linked_event = [{'activityID': '2013-05-04T04:52:00-IPS-001'}]
# for event in example_linked_event:
#     activity_id = extract_activityID_from_dict(event)
#     print(f"Extracted activityID: {activity_id}")


In [14]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
expanded_df.loc[:, 'GST_ActivityID'] = expanded_df['linkedEvents'].apply(lambda x : extract_activityID_from_dict(x))

# Print the DataFrame with the new 'GST_ActivityID' column
expanded_df.head()


,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001


In [15]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
expanded_cme_df_cleaned = expanded_df.dropna(subset=['GST_ActivityID'])
expanded_cme_df_cleaned.head()

,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001


In [16]:
# print out the datatype of each column in this DataFrame:
print(expanded_cme_df_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   activityID      1714 non-null   object
 1   startTime       1714 non-null   object
 2   linkedEvents    1714 non-null   object
 3   GST_ActivityID  1714 non-null   object
dtypes: object(4)
memory usage: 53.7+ KB
None


In [17]:
# Convert the 'GST_ActivityID' column to string format 
expanded_cme_df_cleaned['GST_ActivityID'] = expanded_cme_df_cleaned['GST_ActivityID'].astype(str)

# Convert startTime to datetime format  
expanded_cme_df_cleaned['startTime'] = pd.to_datetime(expanded_cme_df_cleaned['startTime'])

# Rename startTime to startTime_CME and activityID to cmeID
expanded_cme_df_cleaned.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'}, inplace=True)

# Drop linkedEvents
expanded_cme_df_cleaned.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
print(expanded_cme_df_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           1714 non-null   object             
 1   startTime_CME   1714 non-null   datetime64[ns, UTC]
 2   GST_ActivityID  1714 non-null   object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 40.3+ KB
None


In [38]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
cme_df_gst = expanded_cme_df_cleaned[expanded_cme_df_cleaned['GST_ActivityID'].str.contains('GST')]
cme_df_gst.head()


,cmeID,startTime_CME,GST_ActivityID
21,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
48,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
90,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
148,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
151,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001


### GST Data

In [ ]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
# https://api.nasa.gov/DONKI/GST?startDate=yyyy-MM-dd&endDate=yyyy-MM-dd&api_key=DEMO_KEY
query_url = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={KEY}"


In [ ]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(query_url)
gst_response

In [ ]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[0], indent=4))

In [ ]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.DataFrame(gst_json)

# Keep only the columns: activityID, startTime, linkedEvents
gst_df = gst_df[['gstID', 'startTime', 'linkedEvents']]
gst_df.head()

In [ ]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df_cleaned = gst_df.dropna(subset=['linkedEvents'])
gst_df_cleaned.head()

In [ ]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df_exploded = gst_df_cleaned.explode('linkedEvents')
gst_df_exploded.reset_index(drop=True, inplace=True)
gst_df_exploded.dropna(inplace=True)
gst_df_exploded.head()

In [ ]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_df_exploded['CME_ActivityID'] = gst_df_exploded['linkedEvents'].apply(
    lambda x: extract_activityID_from_dict(x) if isinstance(x, dict) else None
)
# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df_final = gst_df_exploded.dropna(subset=['CME_ActivityID'])
gst_df_final.head()

In [ ]:
# Convert the 'CME_ActivityID' column to string format 
gst_df_final['CME_ActivityID'] = gst_df_final['CME_ActivityID'].astype(str)

# Convert the 'gstID' column to string format 
gst_df_final['gstID'] = gst_df_final['gstID'].astype(str)

# Convert startTime to datetime format  
gst_df_final['startTime'] = pd.to_datetime(gst_df_final['startTime'])

# Rename startTime to startTime_GST 
gst_df_final.rename(columns={'startTime': 'startTime_GST'}, inplace=True)

# Drop linkedEvents
gst_df_final.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
gst_df_final.info()

In [ ]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
gst_df_cme_related = gst_df_final[gst_df_final['CME_ActivityID'].str.contains('CME', na=False)]
gst_df_cme_related.head()

### Merge both datatsets

In [ ]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = gst_df_cme_related.merge(
    cme_df_gst, 
    left_on='CME_ActivityID', 
    right_on='GST_ActivityID', 
    how='left'
)

pd.merge(left_df, right_df, left_on=[cols in left_df], right_on=[cols in right_df])

In [ ]:
# Verify that the new DataFrame has the same number of rows as cme and gst
merged_df.info()

### Computing the time it takes for a CME to cause a GST

In [ ]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.


In [ ]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 


### Exporting data in csv format

In [ ]:
# Export data to CSV without the index
